
# Lab 3: Using Mirror's Features
## Using Mirror feature on volumes for disaster recovery or development use.

### With Data Fabric Mirroring you can:

- Mirror data from one cluster to another for disaster recovery (not showcased here today)
- Mirror data localy on one cluster for recovery, development, testing or data validation needs.
- Schedules mirrors or run them on demand.
- Promote mirrors out of read-only mode to read-write

<p align="center">
  <img src="Pictures/edf-mirror-overview.png" width="600" height="400" alt="Data Fabric Snapshots">                                                    
  
</p>

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Data-Fabric101'] }}

echo "We are now connected to the appliance"

#Variables used by scripts
#password="{{ PASSSTU }}" # your Data Fabric login credentials - password - 
#apiserver_host="{{ DATAFABRICAPISERVERIP }}" # API Server for the Data Fabric Cluster
#ad_group="{{ DATAFABRICADGROUP }}" # LDAP Domain Group for Data Fabric users
#vol_name="{{ DATAFABRICVOL }}" # suffix for student volume name
#snapvol_name="{{ DATAFABRICSNAPVOL }}" # suffix for student snap volume name
#cluster_name="{{ DATAFABRICCLUSTERAME }}" # Data Fabric Cluster Name
#src_vol_name="{{ DATAFABRICVOLSRCVOLNAME }}" # suffix for student source mirror volume name
#dst_vol_name="{{ DATAFABRICVOLDSTVOLNAME }}"  # suffix for student destination mirror volume name
#secvol_name="{{ DATAFABRICSECVOL }}"  # suffix for student volume name used for security comparison / user access 
#sec_policy="student{{ STDID }}-policy" #security policy to allow r/w access to volumes

echo "your operation context is:" student{{ STDID }} "with Password:" {{ PASSSTU }}

## 1. Creating a source volume that we will use to mirror from.

#### Background:

You have been given a user that is allowed to create and modify volumes on the cluster.  Data Fabric volumes can be mounted so they are accessable from NFS, HDFS or POSIX commands (like the bash kernel shell in these labs).  For security reasons, you can only mount volumes in the /user directory.

Fist let's create a volume to create mirror's on:

In [ ]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/create?name='$username-$src_vol_name'&path=/user/'$username-$src_vol_name'' --user $username:$password

## 2. Create a mirror volume from your source volume.

Now let's create a local mirror volume from our source volume we made in the previous step:

In [ ]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/create?name='$username-$dst_vol_name'&path=/user/'$username-$dst_vol_name'&type=mirror&source='$username-$src_vol_name'@'$cluster_name'' --user $username:$password

## 3. Create data within your source volume

Now let's create some data in our source volume so we can verify that the mirror operation worked:

In [ ]:
source ~/.bashrc
echo "## Create a file in your source volume that we will mirror/replicate in a later step.##"
echo "$username-$vol_name source volume update $(date)" >> /mapr/$cluster_name/user/$username-$src_vol_name/$username-source-volume-file1
echo "## Verify the file you created in in your directory.##"
ls /mapr/$cluster_name/user/$username-$src_vol_name
echo "## Verify the file has the contents you added.##"
cat /mapr/$cluster_name/user/$username-$src_vol_name/$username'-'source-volume-file1

## 4. Start the mirroring process

By default a target or destination mirror volume is mounted as read-only (ro).  Also by default the mirror is not started until you manually start it or apply a schedule to the mirror.  In this example we will be starting it manually.

**Use `CTRL+Enter` on this cell to run it again in case mirroring not done yet.**

In [ ]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/mirror/start?name='$username-$dst_vol_name'' --user $username:$password

This lab is a short term setup of managing data so we will not cover setting up schedules and applying them to snapshot's or to mirrors.  For more details on creating schedules see: [Creating Schedules on Ezmeral Data Fabric](https://docs.datafabric.hpe.com/62/ReferenceGuide/schedule.html)

## 5. Verify your source data is now within your destination volume

Now let's verify the data we wrote in our previous steps in our source volume is now in the destination volume and that our mirror operation we started worked:

In [ ]:
source ~/.bashrc
echo "## List out the directory contents on our destination volume.##"
ls /mapr/$cluster_name/user/$username-$dst_vol_name

Now let's write some data to your new destination volume that was mirrored from the source volume:


In [ ]:
source ~/.bashrc
echo "##Let's try and write some data to our destination volume...##"
echo "$username-$vol_name destination volume update $(date)" >> /mapr/$cluster_name/user/$username-$dst_vol_name/$username-source-volume-file1
echo "##List out the destination volume directory.##"
ls /mapr/$cluster_name/user/$username-$dst_vol_name
echo "##Verify the file we wrote is on the destination volume directory.##"
cat /mapr/$cluster_name/user/$username-$dst_vol_name/$username-source-volume-file1

**Notice the update to the file did not have the “destination volume update” appended to the file.** This error is to be expected as by default destination volumes that are currently being mirrored are mounted in read-only mode (ro). Let's break the mirror and mount the volume in read-write (rw) mode:

## 6. Promoting destination volumes to read-write

First let's ensure the mirror process has stoped:

In [ ]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/mirror/stop?name='$username-$dst_vol_name'' --user $username:$password

Now we can change the volume mount to read-write:

In [ ]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/modify?name='$username-$dst_vol_name'&type=rw' --user $username:$password

Finally we can verify that the new destination volume has been mounted read-write (rw) by writing new data to it:

In [ ]:
source ~/.bashrc
echo "##Now let's try and write some data to our destination volume...##"
echo "$username-$vol_name destination volume update $(date)" >> /mapr/$cluster_name/user/$username-$dst_vol_name/$username-source-volume-file1
echo "##List out the destination volume directory.##"
ls /mapr/$cluster_name/user/$username-$dst_vol_name
echo "##Verify the file we wrote is on the destination volume directory.##"
cat /mapr/$cluster_name/user/$username-$dst_vol_name/$username-source-volume-file1

You will notice that our file now has the “destination volume update” appended to the file.  This confirms that the destination mirror volume is mounted and working in read-write (rw) mode.

For additional information on using HPE Ezmeral Data Fabric Mirror's Features see: [Mirror Volumes](https://docs.datafabric.hpe.com/62/AdministratorGuide/MirrorVolumes-Mirroring.html)

## 7. Removing mirrors
### Let's cleanup now

 1. Remove the Mirror  volumes
 2. Remove the Source Volume
 
 **Note the `&force=1 here`.  Because Mirror volumes actually use snapshots. It will error out saying there are snapshots you need to remove first. This will just force a removal so we don’t have to remove the snapshots**

In [ ]:
source ~/.bashrc

echo "Deleting dstmirror snapshot"
snapiddstmirror=$(curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/list'| jq -r | grep -B 12 -A 2 ''$username-$dst_vol_name'' | grep snapshotid | cut -c22-30)
echo $snapiddstmirror
curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/remove?snapshots='$snapiddstmirror

echo "Deleting srcmirror snapshot"
snapidsrcmirror=$(curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/list'| jq -r | grep -B 12 -A 2 ''$username-$src_vol_name'' | grep snapshotid | cut -c22-30)
echo $snapidsrcmirror
curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/remove?snapshots='$snapidsrcmirror

echo "Removing the Mirror Volume"
curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/remove?name='$username-$dst_vol_name -force

echo "Removing the Source Volume"
curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/remove?name='$username-$src_vol_name -force


In [ ]:
%logout 

## Mirror's Summary

In this example we:

1. Created a new source volume
2. Wrote data to it
3. Setup a new destination mirror volume
4. Started the mirror process on the destination volume.
5. And then promoted that mirror to be it's own volume and wrote new data to it.

# Using Access Security Settings and Policies  on your data
You can now move to the conclusion of this workshop !

* [Lab 4 - Using Access Security Settings and Policies](4-WKSHP-App-Security-Policies.ipynb)